In [ ]:
import os
os.environ['PROJ_LIB'] = '/usr/local/anaconda3/share/proj'
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from matplotlib import colors
import glob
import math
from scipy.optimize import curve_fit

In [ ]:
def yearly_rainfall ( _year ):
    _csvfiles = glob.glob ( str ( int ( _year ) ) + "/*.csv" )
    print ( len ( _csvfiles ) )
    _cleaned = [ x for x in _csvfiles if "ly" not in x ]
    print ( len ( _cleaned ) ) #show number of .csv files
    return ( _cleaned )

In [ ]:
def csv2index ( _csv ):
    _index = [ int ( i [ 5 : -4 ] ) for i in _csv ]
    return ( _index )

In [ ]:
def remove_outliers ( _index, _outliers ):
    _clean = [ i for i in _index if i not in _outliers ]
    return ( _clean )

In [ ]:
def station_loc ( _path ):
    _station_data = pd.read_csv ( _path, sep = ';|\t', header = None, engine = 'python' )
    lat = float ( _station_data.iloc [ 1, 1 ] )
    lon = float ( _station_data.iloc [ 2, 1 ] )
    return ( lat, lon )

In [ ]:
def yearly_mean ( _year, _index, _limit = 4064 ):
    _rainfall = pd.read_csv ( str ( _year ) + '/' + str ( _index ) + '.csv' )
    _clean = _rainfall [ _rainfall [ 'Rainfall(mm)' ] <= _limit ]
    return ( _clean [ 'Rainfall(mm)' ].mean ( ) )

In [ ]:
def yearly_reliability ( _year, _index, _limit = 4064, _hourlyexpected = 4 ):
    _rainfall = pd.read_csv ( str ( _year ) + '/' + str ( _index ) + '.csv' )
    _clean = _rainfall [ _rainfall [ 'Rainfall(mm)' ] <= _limit ]
    _annualexpected = _hourlyexpected * 24 * 365
    _reliability = len ( _clean ) / _annualexpected
    if ( _reliability == 1.0 ):
        print ( 'Perfect station: ', _index )
    if ( _reliability > 1.0 ):
        print ( _index )
        _reliability = 1.0
    return ( _reliability )

In [ ]:
def distance ( _lat1, _lon1, _lat2, _lon2 ):
    _R = 6377.083
    
    _dLat = math.radians ( _lat2 - _lat1 )
    _dLon = math.radians ( _lon2 - _lon1 )
    _lat1 = math.radians ( _lat1 )
    _lat2 = math.radians ( _lat2 )
 
    _a = math.sin ( _dLat / 2.0 ) ** 2.0 + math.cos ( _lat1 ) * math.cos ( _lat2 ) * math.sin ( _dLon / 2.0 ) ** 2.0
    _c = 2 * math.asin ( math.sqrt ( _a ) )
 
    return ( abs ( _R * _c ) )

In [ ]:
def exponential_variogram ( h, c0, a0 ):
    var = c0 * ( 1 - np.exp ( -h / a0 ) )
    return ( var )

In [ ]:
outliers2013 = [ 611, 486, 425, 563, 955 ]
outliers2014 = [ 425, 908, 877, 884, 149 ]
outliers2015 = [ 934, 809, 685, 400, 307, 884, 88 ]
outliers2016 = [ 355, 291, 657, 884, 853, 630, 344, 254 ]
outliers2017 = [ 608, 923, 166, 422, 73, 559, 52, 853, 917, 148, 827, 925 ]
outliers2018 = [ 322, 166, 744, 904, 41, 200, 269, 115, 248, 62 ]

In [ ]:
year = 2017
stations = yearly_rainfall ( year )
index = csv2index ( stations )
clean = remove_outliers ( index, outliers2017 )

In [ ]:
index_dict = pd.read_csv ( 'FilePathDictionary.csv', names = [ 'Index', 'Station' ], header = None, skiprows = 1 )
index_dict = index_dict [ index_dict [ 'Index' ].isin ( clean ) ]

In [ ]:
_stations = index_dict [ 'Station' ].tolist ( )
stations_lat = [ ]
stations_lon = [ ]
print ( 'Stations with missing locations' )
for i in _stations:
    _loc = station_loc ( i )
    if ( math.isnan ( _loc [ 0 ] ) or math.isnan ( _loc [ 1 ] ) ):
        print ( i )
    stations_lat.append ( _loc [ 0 ] )
    stations_lon.append ( _loc [ 1 ] )

In [ ]:
index_loc = pd.DataFrame ( { 'Index' : index_dict [ 'Index' ], 'Station' : _stations, 'Latitude' : stations_lat, 'Longitude' : stations_lon } )
index_loc = index_loc.dropna ( )

In [ ]:
params = [ ]
for i in range ( len ( index_loc ) ):
    _ref_lat = index_loc [ 'Latitude' ].iloc [ i ]
    _ref_lon = index_loc [ 'Longitude' ].iloc [ i ]
    _ref_rainfall = index_loc [ 'Mean(mm)' ].iloc [ i ]
    _distances = [ ]
    _rainfalldev = [ ]
    for j in range ( len ( index_loc ) ):
        _station_lat = index_loc [ 'Latitude' ].iloc [ j ]
        _station_lon = index_loc [ 'Longitude' ].iloc [ j ]
        _distance = distance ( _ref_lat, _ref_lon, _station_lat, _station_lon )
        _distances.append ( _distance )
        _station_rainfall = index_loc [ 'Mean(mm)' ].iloc [ j ]
        _rainfalldev.append ( ( ( _ref_rainfall - _station_rainfall ) ** 2.0 ) / 2.0 )
    _rainfalldev_mean = sum ( _rainfalldev ) / len ( _rainfalldev )
    _popt, _pcov = curve_fit ( exponential_variogram, _distances, _rainfalldev, bounds = ( [ 0, 0 ], [ _rainfalldev_mean , np.inf ] ) )
    params.append ( _popt )

In [ ]:
c0 = [ i [ 0 ] for i in params ]
a0 = [ i [ 1 ] for i in params ]

In [ ]:
index_loc [ 'c0' ] = c0
index_loc [ 'a0' ] = a0

In [ ]:
index_loc.to_csv ( str ( year ) + 'StationInfluence.csv', index = None )